<a href="https://colab.research.google.com/github/JakeSiewJK64/colabNotebooks/blob/master/iris_flower_classification_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing tensorflow
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

## about our data:
Objective: to predict
*   Setosa
*   Versicolor
*   Virginica

Information we are given:
*   Sepal Length
*   Sepal width
*   Petal Length
*   Petal Width

In [12]:
# importing our data using Tensorflow's 
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file('iris_testing.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

In [25]:
# declaring the column names to read from
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginia']

In [28]:
# storing datasets into a pandas dataframe
train = pd.read_csv(train_path, names= CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

In [31]:
test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


## Train Test Split

In [32]:
train_y = train.pop("Species")
test_y = test.pop("Species")
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


## Writing the input function

In [40]:
def input_fn(features, labels, training = True, batch_size = 256):
  # convert the inputs into a dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

## Creating the feature column

In [43]:
# feature columns describe 
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key = key))

In [44]:
my_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## building our model
- ```DNNClassifier```
- ```LinearClassifier```

In [46]:
# building our DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(feature_columns = my_feature_columns, hidden_units = [30, 10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpqmsj9va1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training the model

In [55]:
# the lambda is to avoid creating an inner function
classifier.train(input_fn = lambda: input_fn(train, train_y, training = True), steps = 15000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqmsj9va1/model.ckpt-15051
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 15051...
INFO:tensorflow:Saving checkpoints for 15051 into /tmp/tmpqmsj9va1/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 15051...
INFO:tensorflow:loss = 0.24440637, step = 15051
INFO:tensorflow:global_step/sec: 444.229
INFO:tensorflow:loss = 0.25161994, step = 15151 (0.227 sec)
INFO:tensorflow:global_step/sec: 508.883
INFO:tensorflow:loss = 0.23808728, step = 15251 (0.198 sec)
INFO:tensorflow:global_step/sec: 577.788
INFO:tensorflow:loss = 0.24263838, step = 15351 (0.170 sec)
INFO:tensorflow:global_step/sec: 583.649
INFO:tensorflow:loss = 0.24601823, step = 15451 (0.171 sec)

## Evaluating the end result

In [56]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test, test_y, training = False))
print("\nAccuracy: {accuracy: 0.3f}\n".format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-24T13:49:06Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqmsj9va1/model.ckpt-30051
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.23156s
INFO:tensorflow:Finished evaluation at 2020-12-24-13:49:06
INFO:tensorflow:Saving dict for global step 30051: accuracy = 0.96666664, average_loss = 0.15742381, global_step = 30051, loss = 0.15742381
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30051: /tmp/tmpqmsj9va1/model.ckpt-30051

Accuracy:  0.967



## Evaluating the result

In [65]:
def input_fn(features, batch_size = 256):
  # convert the inputs to a dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit():
      valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for predict_dict in predictions:
  class_id = predict_dict['class_ids'][0]
  probability = predict_dict['probabilities'][class_id]

  print("Predict is '{}' ({:.1f}%)".format(SPECIES[class_id], 100 * probability))

   
  

Please type numeric values as prompted.
SepalLength: 23
SepalLength: 12
SepalLength: 12
SepalLength: 3
SepalLength: 4
SepalLength: 2
SepalLength: .5
SepalWidth: 2
SepalWidth: .4
PetalLength: .5
PetalWidth: .3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqmsj9va1/model.ckpt-30051
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predict is 'Setosa' (47.1%)


In [66]:
# here is some example input and expected classes you can try above
expected = ['Sentosa', 'Versicolor', 'Virginia']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'SepalLength': [1.7, 4.2, 5.4],
    'SepalWidth': [0.5, 1.5, 2.1],
}